In [ ]:
#import argparse
#import os
#import random
#import time
#import sys
import numpy as np
import pandas as pd
#import shutil

import torch
import torch.nn as nn
#import torch.nn.parallel
import torch.optim
#iimport torch.multiprocessing as mp
import torch.utils.data
import torchvision.transforms as transforms
#import torchvision.datasets as datasets
import torchvision.models as models
from PIL import Image
import cv2


class Dataset(torch.utils.data.Dataset):
    def __init__(self, list_IDs, files, ppm, tranform):
        self.filenames = files
        self.list_IDs = list_IDs
        self.ppm = ppm
        self.transform = tranform

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, index):
        ID = self.list_IDs[index]
        cv_in = cv2.imread('/scratch/ab9738/pollution_img/data/'+self.filenames[ID]+'.jpg',1) # color
        pil_in = Image.fromarray(cv_in)
        X = self.transform(pil_in)
        y = self.ppm[ID]
        return X,y

def double_conv(in_channels, out_channels):
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, 3, padding=1),nn.ReLU(inplace=True),nn.Conv2d(out_channels, out_channels, 3, padding=1),nn.ReLU(inplace=True))

class LeUNet(nn.Module):

    def __init__(self):
        super(LeUNet,self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear') #, align_corners=True)
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        self.conv_last = nn.Conv2d(64, 1, 1)
        self.features = nn.Sequential(nn.Conv2d(1, 6, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),nn.Conv2d(6, 16, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),)
        self.avgpool = nn.AdaptiveAvgPool2d((5, 5))
        self.estimator = nn.Sequential(nn.Linear(16*5*5,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x = self.dconv_down4(x)
        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)
        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        x = self.conv_last(x)
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),16*5*5)
        x = self.estimator(x)
        return x

class LeUNetEns(nn.Module):

    def __init__(self):
        super(LeUNetEns,self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear') #, align_corners=True)
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        self.conv_last = nn.Conv2d(64, 1, 1)
        self.features = nn.Sequential(nn.Conv2d(1, 6, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),nn.Conv2d(6, 16, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),)
        self.avgpool = nn.AdaptiveAvgPool2d((5, 5))
        # self.estimator = nn.Sequential(nn.Linear(16*5*5,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x = self.dconv_down4(x)
        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)
        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        x = self.conv_last(x)
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),16*5*5)
        # x = self.estimator(x)
        return x


class ResNetUNet(nn.Module):

    def __init__(self):
        super(ResNetUNet,self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear') #, align_corners=True)
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        self.conv_last = nn.Conv2d(64, 1, 1)
        self.features = models.resnet50()
        # self.avgpool = nn.AdaptiveAvgPool2d((5, 5))
        self.estimator = nn.Sequential(nn.Linear(1000,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x = self.dconv_down4(x)
        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)
        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        x = self.conv_last(x)
        x = torch.cat((x,x,x),1)
        x = self.features(x)
        # x = self.avgpool(x)
        # x = x.view(x.size(0),16*5*5)
        x = self.estimator(x)
        return x


class StandardNet(nn.Module):

    def __init__(self, modelname='resnet101'):
        super(StandardNet, self).__init__()
        self.model = eval("models."+modelname+"()")
        if(modelname=='inception_v3'):
            self.model = eval("models."+modelname+"(aux_logits=False)")
        self.estimator = nn.Sequential(nn.Linear(1000,120),nn.ReLU(inplace=True),nn.Linear(120,84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        x = self.model(x)
        x = self.estimator(x)
        return x

class EnsembleNet(nn.Module):
    def __init__(self):
        super(EnsembleNet, self).__init__()
        self.resnet50_pred = models.resnet50()
        self.vgg16_pred = models.vgg16()
        self.inceptionv3_pred = models.inception_v3(aux_logits=False)
        self.estimator = nn.Sequential(nn.Linear(3000,120),nn.ReLU(inplace=True),nn.Linear(120,84),nn.ReLU(inplace=True),nn.Linear(84,1),)

    def forward(self, x):
        a = self.resnet50_pred(x)
        b = self.vgg16_pred(x)
        c = self.inceptionv3_pred(x)
        x = torch.cat((a,b,c),1)
        x = self.estimator(x)
        return x

class EnsembleLeUNet(nn.Module):
    def __init__(self):
        super(EnsembleLeUNet, self).__init__()
        self.resnet50_pred = models.resnet50()
        self.vgg16_pred = models.vgg16()
        self.inceptionv3_pred = models.inception_v3(aux_logits=False)
        self.leunet_pred = LeUNetEns()
        # self.leunet_pred = torch.nn.DataParallel(self.leunet_pred).cuda()
        self.leunet_pred.load_state_dict(torch.load("model_haze_all.pth", map_location='cpu'),strict=False)
        self.estimator = nn.Sequential(nn.Linear(3400,120),nn.ReLU(inplace=True),nn.Linear(120,84),nn.ReLU(inplace=True),nn.Linear(84,1),)

    def forward(self, x):
        a = self.resnet50_pred(x)
        b = self.vgg16_pred(x)
        c = self.inceptionv3_pred(x)
        d = self.leunet_pred(x)
        x = torch.cat((a,b,c,d),1)
        x = self.estimator(x)
        return x

class NewReLU(nn.Module):
    """ Custom ReLU layer for baseline """
    def __init__(self, alpha, beta):
        super().__init__()
        self.alpha, self.beta = alpha, beta

    def forward(self, x):
        x = self.alpha*max(x,0) + min(0,x)*self.beta/x+1e-7
        return x

class EPAPLN(nn.Module):

    def __init__(self):
        super(EPAPLN, self).__init__()
        self.conv1 = nn.Conv2d(3,32,3)
        self.conv2 = nn.Conv2d(32,32,3)
        self.maxpool = nn.MaxPool2d(3, 2)
        self.drop1 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(32,64,3)
        self.conv4 = nn.Conv2d(64,64,3)
        self.conv5 = nn.Conv2d(64,64,3)
        self.avgpool = nn.AvgPool2d(3, 2)
        self.drop2 = nn.Dropout(0.5)
        self.conv6 = nn.Conv2d(64,96,3)
        self.conv7 = nn.Conv2d(96,96,3)
        self.conv8 = nn.Conv2d(96,96,3)
        self.conv9 = nn.Conv2d(96,96,3)
        self.activation = nn.ReLU()
        self.adaptivepool = nn.AdaptiveAvgPool2d((2,2))
        self.estimator = nn.Sequential(nn.Linear(96*2*2,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.maxpool(x)
        x = self.drop1(x)
        x = self.conv3(x)
        x = self.activation(x)
        x = self.conv4(x)
        x = self.activation(x)
        x = self.conv5(x)
        x = self.activation(x)
        x = self.avgpool(x)
        x = self.drop2(x)
        x = self.conv6(x)
        x = self.activation(x)
        x = self.conv7(x)
        x = self.activation(x)
        x = self.conv8(x)
        x = self.activation(x)
        x = self.conv9(x)
        x = self.activation(x)
        x = self.adaptivepool(x)
        x = x.view(x.size(0),96*2*2)
        x = self.estimator(x)

        return x


def train(train_loader,model,criterion,optimizer):
    model.train()
    total_loss = 0.0
    epoch_samples = 0
    for x, y in train_loader:
        epoch_samples += x.size(0)
        y = y.float()
        x = x.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)

        x_var = torch.autograd.Variable(x)
        y_var = torch.autograd.Variable(y)

        yhat = model(x_var)
        loss = criterion(yhat.squeeze(),y_var)
        total_loss += loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return (total_loss/epoch_samples)


def val(val_loader,model,criterion):
    model.eval()
    total_loss = 0.0
    epoch_samples = 0
    #with torch.no_grad():
    for x, y in val_loader:
        epoch_samples += x.size(0)
        y = y.float()
        x = x.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)

        x_var = torch.autograd.Variable(x)
        y_var = torch.autograd.Variable(y)

        yhat = model(x_var)
        loss = criterion(yhat.squeeze(),y_var)
        total_loss += loss.data.item()

    return (total_loss/epoch_samples)

def freeze_weights_leunet(model):
    model.module.dconv_down1[0].weight.requires_grad, model.module.dconv_down1[0].bias.requires_grad = False, False
    model.module.dconv_down2[0].weight.requires_grad, model.module.dconv_down2[0].bias.requires_grad = False, False
    model.module.dconv_down3[0].weight.requires_grad, model.module.dconv_down3[0].bias.requires_grad = False, False
    model.module.dconv_down4[0].weight.requires_grad, model.module.dconv_down4[0].bias.requires_grad = False, False
    model.module.dconv_up1[0].weight.requires_grad, model.module.dconv_up1[0].bias.requires_grad = False, False
    model.module.dconv_up2[0].weight.requires_grad, model.module.dconv_up2[0].bias.requires_grad = False, False
    model.module.dconv_up3[0].weight.requires_grad, model.module.dconv_up3[0].bias.requires_grad = False, False
    model.module.dconv_down1[2].weight.requires_grad, model.module.dconv_down1[2].bias.requires_grad = False, False
    model.module.dconv_down2[2].weight.requires_grad, model.module.dconv_down2[2].bias.requires_grad = False, False
    model.module.dconv_down3[2].weight.requires_grad, model.module.dconv_down3[2].bias.requires_grad = False, False
    model.module.dconv_down4[2].weight.requires_grad, model.module.dconv_down4[2].bias.requires_grad = False, False
    model.module.dconv_up1[2].weight.requires_grad, model.module.dconv_up1[2].bias.requires_grad = False, False
    model.module.dconv_up2[2].weight.requires_grad, model.module.dconv_up2[2].bias.requires_grad = False, False
    model.module.dconv_up3[2].weight.requires_grad, model.module.dconv_up3[2].bias.requires_grad = False, False
    model.module.conv_last.weight.requires_grad, model.module.conv_last.bias.requires_grad = False, False
    return model

def main():
    data = pd.read_csv('../train_data.csv')
    data_train = data.sample(frac=0.8,random_state=17)
    data_val = data.loc[~data.index.isin(data_train.index)]
    files_train = list(data_train['filename'])
    files_val = list(data_val['filename'])
    ppm_train = list(data_train['ppm'])
    ppm_val = list(data_val['ppm'])
    ids_train = [i for i in range(len(files_train))]
    ids_val = [i for i in range(len(files_val))]
    data = None
    data_train = None
    data_val = None
#     model = LeUNet()
#     model = ResNetUNet()
#     model = torch.nn.DataParallel(model).cuda()
#     model.load_state_dict(torch.load("model_haze_all.pth"),strict=False) # on GPU
#     model = StandardNet('resnet50').cuda()
#     model = StandardNet('vgg16').cuda()
    model = StandardNet('inception_v3').cuda()
#     model = EPAPLN().cuda()
#     model = EnsembleNet().cuda()  
#     model = EnsembleLeUNet().cuda()  

    criterion = nn.MSELoss().cuda()
    # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=1e-4)
    # model = freeze_weights_leunet(model)

    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    train_dataset = Dataset(ids_train, files_train, ppm_train, transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),])) # normalize
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=12)
    val_dataset = Dataset(ids_val, files_val, ppm_val, transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),]))
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=12)

    best_loss = 1e5
    for epoch in range(5000):
        train_loss = train(train_loader,model,criterion,optimizer)
        val_loss = val(val_loader,model,criterion)
        print('Epoch: %d, MSE train set: %.8f' % (epoch+1, train_loss))
        print('Epoch: %d, MSE val set: %.8f\n' % (epoch+1, val_loss))
        if val_loss < best_loss:
#             torch.save(model.state_dict(),'inception_pm_train.pth')
            best_loss = val_loss

if __name__ == "__main__":
    main()




/scratch/ab9738/pollution_img/env_pol/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Epoch: 1, MSE train set: 343.42850813
Epoch: 1, MSE val set: 311.08689699

Epoch: 2, MSE train set: 289.99379124
Epoch: 2, MSE val set: 243.46678663

Epoch: 3, MSE train set: 175.00705324
Epoch: 3, MSE val set: 89.50927905

Epoch: 4, MSE train set: 72.04649366
Epoch: 4, MSE val set: 72.34485461

Epoch: 5, MSE train set: 57.78322166
Epoch: 5, MSE val set: 52.92154550

Epoch: 6, MSE train set: 50.07916207
Epoch: 6, MSE val set: 51.01158302

Epoch: 7, MSE train set: 43.53920534
Epoch: 7, MSE val set: 52.04837484

Epoch: 8, MSE train set: 37.14970678
Epoch: 8, MSE val set: 58.89543269

Epoch: 9, MSE train set: 32.66791400
Epoch: 9, MSE val set: 97.17304448

Epoch: 10, MSE train set: 24.94913556
Epoch: 10, MSE val set: 47.07009824

Epoch: 11, MSE train set: 24.88776817
Epoch: 11, MSE val set: 47.24741348

Epoch: 12, MSE train set: 22.69370966
Epoch: 12, MSE val set: 48.76162442

Epoch: 13, MSE train set: 23.84764251
Epoch: 13, MSE val set: 52.47259137

Epoch: 14, MSE train set: 19.71624671


Epoch: 112, MSE train set: 3.37500899
Epoch: 112, MSE val set: 42.66921293

Epoch: 113, MSE train set: 5.97423021
Epoch: 113, MSE val set: 38.95107541

Epoch: 114, MSE train set: 4.67923123
Epoch: 114, MSE val set: 34.46058367

Epoch: 115, MSE train set: 5.32003425
Epoch: 115, MSE val set: 36.17355155

Epoch: 116, MSE train set: 5.23705558
Epoch: 116, MSE val set: 35.12901509

Epoch: 117, MSE train set: 5.74506056
Epoch: 117, MSE val set: 38.73428665

Epoch: 118, MSE train set: 5.65220289
Epoch: 118, MSE val set: 36.16436213

Epoch: 119, MSE train set: 7.09777173
Epoch: 119, MSE val set: 39.76192490

Epoch: 120, MSE train set: 5.63447431
Epoch: 120, MSE val set: 36.55123752

Epoch: 121, MSE train set: 5.16159570
Epoch: 121, MSE val set: 37.43535822

Epoch: 122, MSE train set: 3.06648265
Epoch: 122, MSE val set: 36.20667101

Epoch: 123, MSE train set: 4.56174147
Epoch: 123, MSE val set: 38.37386594

Epoch: 124, MSE train set: 5.49605142
Epoch: 124, MSE val set: 37.17621831

Epoch: 125, 

Epoch: 220, MSE train set: 5.43866611
Epoch: 220, MSE val set: 34.85728316

Epoch: 221, MSE train set: 3.70442767
Epoch: 221, MSE val set: 35.06387863

Epoch: 222, MSE train set: 3.10114345
Epoch: 222, MSE val set: 33.04274467

Epoch: 223, MSE train set: 2.83114783
Epoch: 223, MSE val set: 36.59305557

Epoch: 224, MSE train set: 4.60389376
Epoch: 224, MSE val set: 33.23486840

Epoch: 225, MSE train set: 4.18923549
Epoch: 225, MSE val set: 37.72725822

Epoch: 226, MSE train set: 3.79846556
Epoch: 226, MSE val set: 34.49928892

Epoch: 227, MSE train set: 3.63438794
Epoch: 227, MSE val set: 33.97405750

Epoch: 228, MSE train set: 4.26092892
Epoch: 228, MSE val set: 37.36290309

Epoch: 229, MSE train set: 4.25055270
Epoch: 229, MSE val set: 33.10464584

Epoch: 230, MSE train set: 4.96222596
Epoch: 230, MSE val set: 32.78140024

Epoch: 231, MSE train set: 2.85053013
Epoch: 231, MSE val set: 35.78742351

Epoch: 232, MSE train set: 3.58660656
Epoch: 232, MSE val set: 34.41670366

Epoch: 233, 

Epoch: 328, MSE train set: 3.67774876
Epoch: 328, MSE val set: 33.77394499

Epoch: 329, MSE train set: 3.38083574
Epoch: 329, MSE val set: 35.63989599

Epoch: 330, MSE train set: 2.84194855
Epoch: 330, MSE val set: 33.38757538

Epoch: 331, MSE train set: 3.70933546
Epoch: 331, MSE val set: 34.10035375

Epoch: 332, MSE train set: 2.16769056
Epoch: 332, MSE val set: 32.24767512

Epoch: 333, MSE train set: 4.02095715
Epoch: 333, MSE val set: 32.31924759

Epoch: 334, MSE train set: 2.75926161
Epoch: 334, MSE val set: 31.53404823

Epoch: 335, MSE train set: 2.75849376
Epoch: 335, MSE val set: 32.04232255

Epoch: 336, MSE train set: 3.58173325
Epoch: 336, MSE val set: 31.86633984

Epoch: 337, MSE train set: 4.10536169
Epoch: 337, MSE val set: 32.01655248

Epoch: 338, MSE train set: 3.95886784
Epoch: 338, MSE val set: 34.57932180

Epoch: 339, MSE train set: 2.60999169
Epoch: 339, MSE val set: 31.91632293

Epoch: 340, MSE train set: 2.87507585
Epoch: 340, MSE val set: 34.35140287

Epoch: 341, 

...

Epoch: 3202, MSE train set: 0.27223057
Epoch: 3202, MSE val set: 29.15595251

Epoch: 3203, MSE train set: 0.33305266
Epoch: 3203, MSE val set: 29.32461612

Epoch: 3204, MSE train set: 0.33612661
Epoch: 3204, MSE val set: 29.93336028

Epoch: 3205, MSE train set: 0.27812973
Epoch: 3205, MSE val set: 29.46852507

Epoch: 3206, MSE train set: 0.26320721
Epoch: 3206, MSE val set: 28.82817110

Epoch: 3207, MSE train set: 0.25506057
Epoch: 3207, MSE val set: 29.54528979

Epoch: 3208, MSE train set: 0.25296880
Epoch: 3208, MSE val set: 30.07126729

Epoch: 3209, MSE train set: 0.19997884
Epoch: 3209, MSE val set: 28.78797155

Epoch: 3210, MSE train set: 0.23177447
Epoch: 3210, MSE val set: 29.02508673

Epoch: 3211, MSE train set: 0.24592572
Epoch: 3211, MSE val set: 28.98350002

Epoch: 3212, MSE train set: 0.18599189
Epoch: 3212, MSE val set: 29.11357917

Epoch: 3213, MSE train set: 0.27156593
Epoch: 3213, MSE val set: 28.39308006

Epoch: 3214, MSE train set: 0.29003523
Epoch: 3214, MSE val set:

Epoch: 3308, MSE train set: 0.23482004
Epoch: 3308, MSE val set: 28.38887013

Epoch: 3309, MSE train set: 0.26364628
Epoch: 3309, MSE val set: 29.11353222

Epoch: 3310, MSE train set: 0.25256041
Epoch: 3310, MSE val set: 29.36747432

Epoch: 3311, MSE train set: 0.25759518
Epoch: 3311, MSE val set: 29.36390825

Epoch: 3312, MSE train set: 0.21659096
Epoch: 3312, MSE val set: 29.72483943

Epoch: 3313, MSE train set: 0.22577975
Epoch: 3313, MSE val set: 28.97485394

Epoch: 3314, MSE train set: 0.28149742
Epoch: 3314, MSE val set: 29.70538650

Epoch: 3315, MSE train set: 0.38670493
Epoch: 3315, MSE val set: 31.74908234

Epoch: 3316, MSE train set: 0.54594445
Epoch: 3316, MSE val set: 28.59501936

Epoch: 3317, MSE train set: 0.59005884
Epoch: 3317, MSE val set: 31.73001718

Epoch: 3318, MSE train set: 0.46242189
Epoch: 3318, MSE val set: 31.65990272

Epoch: 3319, MSE train set: 0.28357363
Epoch: 3319, MSE val set: 31.55249706

Epoch: 3320, MSE train set: 0.23478870
Epoch: 3320, MSE val set:

Epoch: 3414, MSE train set: 0.33267317
Epoch: 3414, MSE val set: 30.15524356

Epoch: 3415, MSE train set: 0.33837274
Epoch: 3415, MSE val set: 29.95859255

Epoch: 3416, MSE train set: 0.29973442
Epoch: 3416, MSE val set: 30.40392981

Epoch: 3417, MSE train set: 0.27952666
Epoch: 3417, MSE val set: 31.51309610

Epoch: 3418, MSE train set: 0.29380240
Epoch: 3418, MSE val set: 31.17256773

Epoch: 3419, MSE train set: 0.26336622
Epoch: 3419, MSE val set: 31.95970527

Epoch: 3420, MSE train set: 0.33932251
Epoch: 3420, MSE val set: 30.23068942

Epoch: 3421, MSE train set: 0.25271877
Epoch: 3421, MSE val set: 30.65790563

Epoch: 3422, MSE train set: 0.24529001
Epoch: 3422, MSE val set: 30.73585222

Epoch: 3423, MSE train set: 0.19862332
Epoch: 3423, MSE val set: 30.70760746

Epoch: 3424, MSE train set: 0.25700003
Epoch: 3424, MSE val set: 31.36178738

Epoch: 3425, MSE train set: 0.23409489
Epoch: 3425, MSE val set: 31.36004297

Epoch: 3426, MSE train set: 0.26263920
Epoch: 3426, MSE val set:

Epoch: 3520, MSE train set: 0.22405020
Epoch: 3520, MSE val set: 30.52379133

Epoch: 3521, MSE train set: 0.23517527
Epoch: 3521, MSE val set: 31.54411433

Epoch: 3522, MSE train set: 0.24613000
Epoch: 3522, MSE val set: 30.57043969

Epoch: 3523, MSE train set: 0.24180076
Epoch: 3523, MSE val set: 31.39660303

Epoch: 3524, MSE train set: 0.20689011
Epoch: 3524, MSE val set: 31.71759247

Epoch: 3525, MSE train set: 0.24408672
Epoch: 3525, MSE val set: 30.31139411

Epoch: 3526, MSE train set: 0.27429783
Epoch: 3526, MSE val set: 31.85346270

Epoch: 3527, MSE train set: 0.24986589
Epoch: 3527, MSE val set: 31.24665288

Epoch: 3528, MSE train set: 0.21612514
Epoch: 3528, MSE val set: 31.10829942

Epoch: 3529, MSE train set: 0.26023864
Epoch: 3529, MSE val set: 31.47488510

Epoch: 3530, MSE train set: 0.19036659
Epoch: 3530, MSE val set: 31.37782554

Epoch: 3531, MSE train set: 0.18774389
Epoch: 3531, MSE val set: 31.46509771

Epoch: 3532, MSE train set: 0.22014132
Epoch: 3532, MSE val set:

Epoch: 3626, MSE train set: 0.20261663
Epoch: 3626, MSE val set: 30.01123730

Epoch: 3627, MSE train set: 0.23342846
Epoch: 3627, MSE val set: 29.84502662

Epoch: 3628, MSE train set: 0.24776606
Epoch: 3628, MSE val set: 30.96357354

Epoch: 3629, MSE train set: 0.24359413
Epoch: 3629, MSE val set: 30.76141955

Epoch: 3630, MSE train set: 0.26700868
Epoch: 3630, MSE val set: 30.12975562

Epoch: 3631, MSE train set: 0.26111691
Epoch: 3631, MSE val set: 30.44682685

Epoch: 3632, MSE train set: 0.21385369
Epoch: 3632, MSE val set: 29.86177330

Epoch: 3633, MSE train set: 0.25694278
Epoch: 3633, MSE val set: 29.92936397

Epoch: 3634, MSE train set: 0.28038675
Epoch: 3634, MSE val set: 29.22364647

Epoch: 3635, MSE train set: 0.27485211
Epoch: 3635, MSE val set: 31.83516538

Epoch: 3636, MSE train set: 0.28776913
Epoch: 3636, MSE val set: 30.15576983

Epoch: 3637, MSE train set: 0.21500685
Epoch: 3637, MSE val set: 30.69850767

Epoch: 3638, MSE train set: 0.25145393
Epoch: 3638, MSE val set: